In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt 
%matplotlib inline 
# It will make plot output appear in the notebook as well as save in the notebook, hence next time when you open, you would be able to see the plot, without re running
plt.rc("font", size=14)
from sklearn.preprocessing import LabelEncoder # Just in case the output variable is in string form
lb = LabelEncoder() # Also if independent variables are in string format, encode those ones as well
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import warnings # Used to ignore warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
RandomForestClassifier() # Place the type bar in the parenthesis of the function and press Shift and double tab
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import NearMiss


In [2]:
data = pd.read_csv('online_shoppers_intention.csv')

In [3]:
data['Revenue'] = lb.fit_transform(data['Revenue'])
data['Month'] = lb.fit_transform(data['Month'])
data['VisitorType'] = lb.fit_transform(data['VisitorType'])

In [4]:
y = data['Revenue']
y

0        0
1        0
2        0
3        0
4        0
        ..
12325    0
12326    0
12327    0
12328    0
12329    0
Name: Revenue, Length: 12330, dtype: int64

In [5]:
x = data.drop(columns = ['Revenue'], inplace = True)

In [6]:
x_train, x_test, y_train, y_test= train_test_split(data,y,test_size=0.3)

In [7]:
sc_x = StandardScaler()
xtrain = sc_x.fit_transform(x_train)
xtest = sc_x.transform(x_test)

In [8]:
sm = SMOTE(random_state = 9)

In [9]:
x_train_SMOT, y_train_SMOT = sm.fit_resample(x_train, y_train)

In [10]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))

  
print("After OverSampling, counts of label '1': {}".format(sum(y_train_SMOT == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_SMOT == 0)))

Before OverSampling, counts of label '1': 1342
Before OverSampling, counts of label '0': 7289 

After OverSampling, counts of label '1': 7289
After OverSampling, counts of label '0': 7289


# Logistic after SMOTE

In [11]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(x_train_SMOT, y_train_SMOT)

LogisticRegression(random_state=0)

In [12]:
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)

In [13]:
print ("Confusion Matrix : \n", cm)

Confusion Matrix : 
 [[2865  268]
 [ 168  398]]


In [14]:
print ("Accuracy : ", accuracy_score(y_test, y_pred)*100)

Accuracy :  88.21303054879697


In [15]:
recall_score(y_test, y_pred, average=None)# Precision

array([0.91445898, 0.70318021])

In [16]:
precision_score(y_test, y_pred, average=None)

array([0.9446093, 0.5975976])

In [17]:
print(classification_report(y_test, y_pred))# You can print this one only, if needed.

              precision    recall  f1-score   support

           0       0.94      0.91      0.93      3133
           1       0.60      0.70      0.65       566

    accuracy                           0.88      3699
   macro avg       0.77      0.81      0.79      3699
weighted avg       0.89      0.88      0.89      3699



# Decision Tree After SMOTE

In [18]:
clf_entropy = DecisionTreeClassifier(
            criterion = "entropy", random_state = 100,
            )

In [19]:
clf_entropy.fit(x_train_SMOT, y_train_SMOT)

DecisionTreeClassifier(criterion='entropy', random_state=100)

In [20]:
y_pred_entropy = clf_entropy.predict(x_test)

In [21]:

print("Confusion Matrix for Entropy: ",confusion_matrix(y_test, y_pred_entropy))  


Confusion Matrix for Entropy:  [[2781  352]
 [ 196  370]]


In [22]:
print ("Accuracy for Entropy ", accuracy_score(y_test,y_pred_entropy)*100)     

Accuracy for Entropy  85.18518518518519


In [23]:
print("Report for Entropy : ",classification_report(y_test, y_pred_entropy))

Report for Entropy :                precision    recall  f1-score   support

           0       0.93      0.89      0.91      3133
           1       0.51      0.65      0.57       566

    accuracy                           0.85      3699
   macro avg       0.72      0.77      0.74      3699
weighted avg       0.87      0.85      0.86      3699



# Randomforest after SMOTE

In [24]:
rf_classifier=RandomForestClassifier(n_estimators = 10).fit(x_train_SMOT,y_train_SMOT)
prediction=rf_classifier.predict(x_test)

In [25]:
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[2903  230]
 [ 173  393]]
0.891051635577183
              precision    recall  f1-score   support

           0       0.94      0.93      0.94      3133
           1       0.63      0.69      0.66       566

    accuracy                           0.89      3699
   macro avg       0.79      0.81      0.80      3699
weighted avg       0.90      0.89      0.89      3699



In [26]:
model=RandomForestClassifier(n_estimators=300,criterion='entropy',
                             max_features='sqrt',min_samples_leaf=10,random_state=100).fit(x_train,y_train)
predictions=model.predict(x_test)
print(confusion_matrix(y_test,predictions))
print(accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

[[3025  108]
 [ 234  332]]
0.9075425790754258
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      3133
           1       0.75      0.59      0.66       566

    accuracy                           0.91      3699
   macro avg       0.84      0.78      0.80      3699
weighted avg       0.90      0.91      0.90      3699



# Near miss algorithm

In [27]:
nr = NearMiss()

In [28]:
x_train_miss, y_train_miss = nr.fit_resample(x_train, y_train)

In [29]:
print("Before Under Sampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before Under Sampling, counts of label '0': {} \n".format(sum(y_train == 0)))

  
print("After Under Sampling, counts of label '1': {}".format(sum(y_train_miss == 1)))
print("After Under Sampling, counts of label '0': {}".format(sum(y_train_miss == 0)))

Before Under Sampling, counts of label '1': 1342
Before Under Sampling, counts of label '0': 7289 

After Under Sampling, counts of label '1': 1342
After Under Sampling, counts of label '0': 1342


# Logistic after Near Miss

In [30]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(x_train_miss, y_train_miss)

LogisticRegression(random_state=0)

In [31]:
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)

In [32]:
print ("Confusion Matrix : \n", cm)

Confusion Matrix : 
 [[1446 1687]
 [  37  529]]


In [33]:
print ("Accuracy : ", accuracy_score(y_test, y_pred)*100)

Accuracy :  53.39280886726142


In [34]:
print(classification_report(y_test, y_pred))# You can print this one only, if needed.

              precision    recall  f1-score   support

           0       0.98      0.46      0.63      3133
           1       0.24      0.93      0.38       566

    accuracy                           0.53      3699
   macro avg       0.61      0.70      0.50      3699
weighted avg       0.86      0.53      0.59      3699



# Decision Tree after near miss


In [35]:
clf_entropy.fit(x_train_miss, y_train_miss)

DecisionTreeClassifier(criterion='entropy', random_state=100)

In [36]:
print ("Accuracy for Entropy ", accuracy_score(y_test,y_pred_entropy)*100)     

Accuracy for Entropy  85.18518518518519


In [37]:
print("Report for Entropy : ",classification_report(y_test, y_pred_entropy))

Report for Entropy :                precision    recall  f1-score   support

           0       0.93      0.89      0.91      3133
           1       0.51      0.65      0.57       566

    accuracy                           0.85      3699
   macro avg       0.72      0.77      0.74      3699
weighted avg       0.87      0.85      0.86      3699



# Random Forest after near miss

In [38]:
rf_classifier=RandomForestClassifier(n_estimators = 10).fit(x_train_miss,y_train_miss)
prediction=rf_classifier.predict(x_test)

In [39]:
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[1376 1757]
 [  23  543]]
0.5187888618545553
              precision    recall  f1-score   support

           0       0.98      0.44      0.61      3133
           1       0.24      0.96      0.38       566

    accuracy                           0.52      3699
   macro avg       0.61      0.70      0.49      3699
weighted avg       0.87      0.52      0.57      3699

